In [1]:
from catboost import CatBoostClassifier, CatBoostRegressor, FeaturesData
from sklearn.model_selection import train_test_split
from database.async_db import asyncHandler as db
from navec import Navec
import numpy as np

# 1. Sentence embedding

In [2]:
async def get_vector_data(id_t, id_f):
    t = [await db.get_vector_by_p_id(i) for i in id_t]
    f = [await db.get_vector_by_p_id(i) for i in id_f]
    return t + f, [1] * len(t) + [0] * len(f)

In [3]:
id_t = [48,115,170,873,2919,1364,380,8986,614,2391,3812,10810]

id_f = [9007,8915,11427,385,5451,9366,5079,3816,3955,2424,3593,49,174,2949,893,120,3843,1382]

x, y = await get_vector_data(id_t, id_f)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

cat = CatBoostClassifier(
    thread_count=8,
    iterations=50,
    random_seed=42,
    learning_rate=0.1,
    custom_loss=['AUC', 'Accuracy', 'PRAUC']
)

cat.fit(
   X_train, y_train,
   eval_set=(X_test, y_test),
   logging_level='Silent',
   plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

# 2. all feaches

In [4]:
async def get_cat_data(id_t, id_f):
    out = await db.get_cat_data_by_list_id(id_t + id_f)
    return out, [1] * len(id_t) + [0] * len(id_f)

In [5]:
id_t = [48,115,170,873,2919,1364,380,8986,614,2391,3812,10810]

id_f = [9007,8915,11427,385,5451,9366,5079,3816,3955,2424,3593,49,174,2949,893,120,3843,1382]

x, y = await get_cat_data(id_t, id_f)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [6]:
cat = CatBoostClassifier(
        thread_count=8,
        iterations=50,
        random_seed=42,
        learning_rate=0.1,
        custom_loss=['AUC', 'Accuracy', 'PRAUC']
    )

cat.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    embedding_features=[0, 9],
    cat_features=[1,2,3,4,5,6,7,8],
    logging_level='Silent',
    plot=True
)

D:\PycharmProjects\RTUITLab_testTask\venv\lib\site-packages\numpy\core\fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))